# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import os

# Define Source

In [2]:
link = 'https://www.autism.org/advice-for-parents/'

# Read 100 questions list

In [3]:
file = 'Excel/' + os.listdir('Excel')[-1]
df = pd.read_excel(file,engine='openpyxl')

df[link]=np.nan
df

,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,NaN
...,...,...,...,...
101,What are some ways that parents can reduce the...,NaN,NaN,NaN
102,Do some families deal with stress better than ...,NaN,NaN,NaN
103,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN
104,What can I do about my children’s stress?,NaN,NaN,NaN


# Scrape QA pairs from website

In [4]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [5]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [6]:
l1=[]
for i in soup.find_all('h3'):
    l1.append(i.text.strip())
l1 = l1[1:]
l1

['What Is Autism',
 'Diagnosis',
 'Onset of Autism: Early Onset vs. Regression',
 'Causes of Autism',
 'Prevalence of Autism',
 'Medical Testing and Treatments',
 'Psychiatric Medications',
 'Rimland and Edelson',
 'Adams',
 'Speech Development',
 'Educational/Behavioral Approaches',
 'RDI',
 'Preparing for the Future',
 'Developing Friendships',
 'State Services',
 'School Programs',
 'Social Security Assistance',
 'Special Needs Trust',
 'Long-term Prognosis',
 'What To Do Next']

In [7]:
# Search the answers for questions
l2 = []
c = 0
for j in range(5,46):
    #print(l1[c])
    c=c+1
    ans = soup.find('div',{'class':'fusion-text fusion-text-{}'.format(j)})
    #print(ans.text)
    #print('*'*100)
    if(ans.text.strip()!='back to menu'):
        l2.append(ans.text)

In [8]:
l2[7:10] = [' '.join(l2[7:10])]
for i in range(len(l2)):
    print(i, l2[i])
    print('*'*100)

0 Autism is a developmental disability that typically involves delays and impairment in social skills, language, and behavior. Autism is a spectrum disorder, meaning that it affects people differently. Some children may have speech, whereas others may have little or no speech. Autism is a very broad spectrum. In profound cases, young children may not interact with others, or treat people as objects. In milder cases it involves difficulty understanding and relating to others, and difficulty understanding other people’s perspectives and emotions. Without appropriate support, children will not develop effective social skills and may speak or behave in ways that create challenges. Very few individuals recover completely from autism without any intervention. The good news is that there are a wide variety of medically appropriate treatment options which may be very helpful. Some treatments may lead to measurab le improvement, whereas other treatments may have little or no effect. Because the

In [9]:
len(l1),len(l2)

(20, 20)

In [10]:
l3 = []
for i in soup.find_all('h4'):
    l3.append(i.text.strip())
l3 = l3[:7]
l3

['What is the difference between Asperger’s Syndrome and Autism?',
 'Common Co-Occurring Conditions in Autism',
 'Applied Behavior Analysis',
 'Speech Therapy',
 'Occupational Therapy',
 'Auditory Interventions',
 'Computer Software']

In [11]:
for i in soup.find_all('h4'):
    x = i.text.strip()
    for j in range(len(l2)):
        if(x in l2[j]):
            l2[j] = l2[j].replace(x,'\n\n'+x)

In [12]:
for i in range(len(l2)):
    print(i,l1[i])
    print('-'*50)
    print(l2[i])
    print('*'*100)

0 What Is Autism
--------------------------------------------------
Autism is a developmental disability that typically involves delays and impairment in social skills, language, and behavior. Autism is a spectrum disorder, meaning that it affects people differently. Some children may have speech, whereas others may have little or no speech. Autism is a very broad spectrum. In profound cases, young children may not interact with others, or treat people as objects. In milder cases it involves difficulty understanding and relating to others, and difficulty understanding other people’s perspectives and emotions. Without appropriate support, children will not develop effective social skills and may speak or behave in ways that create challenges. Very few individuals recover completely from autism without any intervention. The good news is that there are a wide variety of medically appropriate treatment options which may be very helpful. Some treatments may lead to measurab le improvement, 

# Check which questions are similar

In [13]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('autism spectrum disorder','asd',sent) # Compress term
    sent = re.sub('autism','asd',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [14]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [15]:
def sim(li):
    for c1,i in enumerate(li):
        i2 = clean_text(i)
        list1 = i2.split()

        temp1 = 0.0
        temp2 = ''
        temp3 = ''
        temp4 = 0
        temp5 = 0

        for c2,j in enumerate(df['Question']):
            j2 = clean_text(j)
            list2 = j2.split()

            sim = jaccard_similarity(list1, list2)

            if(sim>temp1):
                temp1 = sim
                temp2 = j
                temp3 = i
                temp4 = c2
                temp5 = c1

        if(temp1>0):
            print('Website --> ',temp3,'(Index {})'.format(temp5))
            print('100 questions list --> ',temp2,'(Index {})'.format(temp4))
            print(temp1)
            print('*'*100)

In [16]:
sim(l1)

Website -->  What Is Autism (Index 0)
100 questions list -->  What is Autism? (Index 4)
1.0
****************************************************************************************************
Website -->  Diagnosis (Index 1)
100 questions list -->  Are their tests that are not recommended for the diagnosis or management of autism? (Index 25)
0.2
****************************************************************************************************
Website -->  Onset of Autism: Early Onset vs. Regression (Index 2)
100 questions list -->  What are 16 early signs of autism by 16 months? (Index 3)
0.25
****************************************************************************************************
Website -->  Causes of Autism (Index 3)
100 questions list -->  What is Autism? (Index 4)
0.5
****************************************************************************************************
Website -->  Prevalence of Autism (Index 4)
100 questions list -->  What is Autism? (Index 4)
0.5
**

In [17]:
# Website, 100Q
# 0, 4
# 1, 1
# 2, 3
# 3, 90
# 5, 23
# 10, 40
# 14, 74
# 15, 44
# 16, 70
# 18, 14
# 19, 8

In [18]:
# Directly add answer to existing dataframe
l4 = [[0, 4],
    [1, 1],
    [2, 3],
    [3, 90],
    [5, 23],
    [10, 40],
    [14, 74],
    [15, 44],
    [16, 70],
    [18, 14],
    [19, 8]]

for i in l4:
    df[link].loc[i[1]] = l2[i[0]]

C:\Users\aparg\anaconda3\envs\ASD_scraping\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [19]:
sim(l3)

Website -->  What is the difference between Asperger’s Syndrome and Autism? (Index 0)
100 questions list -->  How can you tell Autism from Asperger’s Syndrome? (Index 10)
0.6
****************************************************************************************************
Website -->  Common Co-Occurring Conditions in Autism (Index 1)
100 questions list -->  What is Autism? (Index 4)
0.25
****************************************************************************************************
Website -->  Applied Behavior Analysis (Index 2)
100 questions list -->  What is Applied Behavioral Analysis? (Index 34)
0.5
****************************************************************************************************
Website -->  Speech Therapy (Index 3)
100 questions list -->  What is chelation therapy? (Index 64)
0.3333333333333333
****************************************************************************************************
Website -->  Occupational Therapy (Index 4)
100 questions 

In [20]:
# 0, 10
# 2, 34

In [21]:
# Directly add answer to existing dataframe
df[link].loc[10] = l2[0].split(l3[0])[1]
df[link].loc[34] = l2[10].split(l3[2])[1]

In [22]:
l1

['What Is Autism',
 'Diagnosis',
 'Onset of Autism: Early Onset vs. Regression',
 'Causes of Autism',
 'Prevalence of Autism',
 'Medical Testing and Treatments',
 'Psychiatric Medications',
 'Rimland and Edelson',
 'Adams',
 'Speech Development',
 'Educational/Behavioral Approaches',
 'RDI',
 'Preparing for the Future',
 'Developing Friendships',
 'State Services',
 'School Programs',
 'Social Security Assistance',
 'Special Needs Trust',
 'Long-term Prognosis',
 'What To Do Next']

In [23]:
# update questions list
f = [i[0] for i in l4]
f = set(range(len(l1)))-set(f)
l1=[l1[i] for i in f]
l1

['Prevalence of Autism',
 'Psychiatric Medications',
 'Rimland and Edelson',
 'Adams',
 'Speech Development',
 'RDI',
 'Preparing for the Future',
 'Developing Friendships',
 'Special Needs Trust']

In [24]:
# select answers to questions in list 'l'
l2 = [l2[i] for i in f]

In [25]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l1,l2),columns=['Question',link])
df2

,Question,https://www.autism.org/advice-for-parents/
0,Prevalence of Autism,There has been a rapid increase in the number ...
1,Psychiatric Medications,The various topics covered in this overview pa...
2,Rimland and Edelson,"ARI has, for many years, encouraged research o..."
3,Adams,Psychiatric medications are not well-tested in...
4,Speech Development,One of the most common questions parents ask i...
5,RDI,This is a method for teaching children how to ...
6,Preparing for the Future,Temple Grandin: “As a person with autism I wan...
7,Developing Friendships,Although young children with autism may seem t...
8,Special Needs Trust,Children who have assets over approximately $2...


In [26]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3.head(10)

,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...
5,What are Screening Tools for Autism?,NaN,Early intervention can make a big difference f...,NaN
6,How Common Is It?,NaN,"For many years a diagnosis of autism was rare,...",NaN
7,What are the Signs of Autism?,NaN,Many signs can indicate that an individual may...,NaN
8,What should I do next?,NaN,When preparing to talk to your medical provide...,"Attend one or more parent support groups, o..."
9,How can you tell Autism from Asperger’s Syndrome?,NaN,NaN,Asperger syndrome was phased out with the int...


In [27]:
df3.shape

(115, 4)

# Save Output

In [28]:
df3.to_excel('Excel/Source2_2.xlsx',index=False)